<div style="text-align: center"><a href="../index.ipynb">LAMMPS Python Tutorials</a></div>

# Example 5: Monte Carlo Relaxation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import random, math

## Setup perfect system

In [ ]:
from lammps import lammps

In [ ]:
L = lammps()
cmd = L.cmd

In [ ]:
cmd.units("lj")
cmd.atom_style("atomic")
cmd.atom_modify("map array sort", 0, 0.0)

cmd.dimension(2)

cmd.lattice("hex", 1.0)
cmd.region("box block", 0, 10, 0, 5, -0.5, 0.5)

cmd.create_box(1, "box")
cmd.create_atoms(1, "box")
cmd.mass(1, 1.0)

cmd.pair_style("lj/cut", 2.5)
cmd.pair_coeff(1, 1, 1.0, 1.0, 2.5)
cmd.pair_modify("shift", "yes")

cmd.neighbor(0.3, "bin")
cmd.neigh_modify("delay", 0, "every", 1, "check", "yes")

In [ ]:
L.ipython.image(zoom=1.6,size=[320,320])

In [ ]:
cmd.run(0, "post", "no")

In [ ]:
emin = L.get_thermo("pe")

In [ ]:
cmd.dump("3 all movie 25 movie.mp4 type type zoom 1.6 adiam 1.0")

## Disorder system

In [ ]:
random.seed(27848)
deltaperturb = 0.2

In [ ]:
pos = L.numpy.extract_atom("x")
for i in range(len(pos)):
    x, y = pos[i][0], pos[i][1]
    dx = deltaperturb * random.uniform(-1, 1)
    dy = deltaperturb * random.uniform(-1, 1)
    pos[i] = (x+dx, y+dy, 0)

In [ ]:
cmd.run(0, "post", "no")

In [ ]:
L.ipython.image(zoom=1.6,size=[320,320])

## Minimize using Monte Carlo moves

In [ ]:
estart = L.get_thermo("pe")
elast = estart

In [ ]:
naccept = 0

In [ ]:
energies = [estart]

In [ ]:
niterations = 3000
deltamove = 0.1
kT = 0.05

In [ ]:
natoms = L.extract_global("natoms")

for i in range(niterations):
    pos = L.numpy.extract_atom("x")
    iatom = random.randrange(0, natoms)
    current_atom = pos[iatom]
    
    x0, y0 = current_atom[0], current_atom[1]
    
    dx = deltamove * random.uniform(-1, 1)
    dy = deltamove * random.uniform(-1, 1)
    
    pos[iatom] = (x0+dx, y0+dy, 0)
    
    cmd.run(1, "pre no post no")
    
    e = L.get_thermo("pe")
    energies.append(e)
    
    if e <= elast:
        naccept += 1
        elast = e
    elif random.random() <= math.exp(natoms*(elast-e)/kT):
        naccept += 1
        elast = e
    else:
        pos[iatom] = (x0, y0, 0)

In [ ]:
plt.xlabel('iteration')
plt.ylabel('potential energy')
plt.plot(energies)

In [ ]:
L.get_thermo("pe")

In [ ]:
emin

In [ ]:
estart

In [ ]:
naccept

In [ ]:
L.ipython.image(zoom=1.6, size=[320,320])

In [ ]:
# close dump file to access it
cmd.undump(3)

In [ ]:
L.ipython.video("movie.mp4")